In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
from sklearn import datasets
import numpy as np
import pandas as pd
import io
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns;

import warnings
warnings.filterwarnings("ignore")


In [5]:
from google.colab import files
uploaded = files.upload()

Saving Churn_Modelling.csv to Churn_Modelling.csv


In [0]:
import io
bankDF=pd.read_csv(io.BytesIO(uploaded['Churn_Modelling.csv']))

In [36]:
bankDF.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [15]:
bankDF.shape


(10000, 14)

In [0]:
bankDF=bankDF.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [17]:
bankDF.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [19]:
bankDF.isnull().count()

CreditScore        10000
Geography          10000
Gender             10000
Age                10000
Tenure             10000
Balance            10000
NumOfProducts      10000
HasCrCard          10000
IsActiveMember     10000
EstimatedSalary    10000
Exited             10000
dtype: int64

In [20]:
bankDF.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [0]:
XDF = pd.get_dummies(bankDF, prefix_sep='_')

In [430]:
XDF['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [43]:
XDF.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [0]:
x=XDF.drop('Exited',axis=1)
y=pd.DataFrame(XDF['Exited'],columns=['Exited'])

In [79]:
x.shape

(10000, 13)

In [80]:
y.head()

,Exited
0,1
1,0
2,1
3,0
4,0


In [0]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20,random_state=1)  

In [0]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  # z-scalar
scaler.fit(X_train) # Compute the mean and std to be used for later scaling.

X_train = scaler.transform(X_train)  # scales the data, Perform standardization by centering and scaling
X_test = scaler.transform(X_test)  

In [306]:
X_train.shape

(8000, 13)

In [307]:
y_train.shape


(8000, 1)

In [0]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()

In [0]:
y_train = pd.DataFrame(onehotencoder.fit_transform(y_train).toarray())
y_test = pd.DataFrame(onehotencoder.fit_transform(y_test).toarray())

In [0]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [0]:
def leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=0.4)

In [0]:
epochs=40
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.3

sgd = tf.keras.optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

In [0]:
model = Sequential()


In [0]:
model.add(Dense(13, activation=leaky_relu, input_shape=(13,)))
model.add(Dense(13, activation=leaky_relu))
model.add(Dense(13, activation=leaky_relu))
model.add(Dense(2,activation='softmax'))


In [0]:
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])


In [359]:
len(model.layers)

4

In [360]:
model.summary()

Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_154 (Dense)            (None, 13)                182       
_________________________________________________________________
dense_155 (Dense)            (None, 13)                182       
_________________________________________________________________
dense_156 (Dense)            (None, 13)                182       
_________________________________________________________________
dense_157 (Dense)            (None, 2)                 28        
Total params: 574
Trainable params: 574
Non-trainable params: 0
_________________________________________________________________


In [361]:
model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test))


Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 1s 99us/sample - loss: 0.4467 - accuracy: 0.8041 - val_loss: 0.4195 - val_accuracy: 0.8095
Epoch 2/100
8000/8000 [==============================] - 0s 54us/sample - loss: 0.4203 - accuracy: 0.8173 - val_loss: 0.4084 - val_accuracy: 0.8230
Epoch 3/100
8000/8000 [==============================] - 0s 58us/sample - loss: 0.4059 - accuracy: 0.8263 - val_loss: 0.3967 - val_accuracy: 0.8325
Epoch 4/100
8000/8000 [==============================] - 0s 56us/sample - loss: 0.3896 - accuracy: 0.8359 - val_loss: 0.3783 - val_accuracy: 0.8440
Epoch 5/100
8000/8000 [==============================] - 0s 56us/sample - loss: 0.3726 - accuracy: 0.8447 - val_loss: 0.3621 - val_accuracy: 0.8495
Epoch 6/100
8000/8000 [==============================] - 0s 56us/sample - loss: 0.3618 - accuracy: 0.8505 - val_loss: 0.3551 - val_accuracy: 0.8600
Epoch 7/100
8000/8000 [==============================] - 0s 56us

In [362]:
score=model.evaluate(X_test,y_test)

2000/2000 [==============================] - 0s 40us/sample - loss: 0.3329 - accuracy: 0.8665


In [363]:
score

[0.33293636202812193, 0.8665]

In [0]:
y_pred=model.predict(X_test)


In [0]:
y_pred[:,1] = (y_pred[:,1] >= 0.5).astype(int)
y_pred[:,0] = (y_pred[:,0] >= 0.5).astype(int)


In [0]:
df=pd.DataFrame(y_pred,columns=['Exited','Non-Exited'])

In [416]:
df.head()

,Exited,Non-Exited
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [417]:
df['Exited'].unique()

array([1., 0.])

In [418]:
df['Non-Exited'].unique()

array([0., 1.])

In [0]:
y_actual=y_test[:][0]
y_predict=df['Exited']

In [0]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 


In [427]:
results = confusion_matrix(y_actual, y_predict) 
print('Confusion Matrix :')
print(results) 
print('Accuracy Score :',accuracy_score(y_actual, y_predict) )
print('Report : ')
print(classification_report(y_actual, y_predict) )

Confusion Matrix :
[[ 208  207]
 [  60 1525]]
Accuracy Score : 0.8665
Report : 
              precision    recall  f1-score   support

         0.0       0.78      0.50      0.61       415
         1.0       0.88      0.96      0.92      1585

    accuracy                           0.87      2000
   macro avg       0.83      0.73      0.76      2000
weighted avg       0.86      0.87      0.86      2000



In [435]:
y_actual.value_counts()

1.0    1585
0.0     415
Name: 0, dtype: int64

we have an accuracy score of 86.65%, loss of 33.2% .